### pull pybaseball data

In [ ]:
import pandas as pd 
from pybaseball import statcast
from pybaseball import statcast_batter
from pybaseball import cache
import os
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Swing Decisons')
cache.enable()

In [4]:
df = statcast('2021-01-01', '2024-12-30')

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


 21%|██▏       | 211/984 [01:19<03:35,  3.58it/s]c:\Users\dalto\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 40%|████      | 396/984 [02:26<01:39,  5.89it/s]c:\Users\dalto\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 40%|████      | 397/984 [02:27<04:19,  2.26it/s]c:\Users\dalto\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a fu

In [11]:
df_keep = df[['pitch_type', 'pitch_name', 'player_name', 'pitcher', 'batter', 'description', 'zone', 'estimated_woba_using_speedangle', 'stand', 'p_throws', 'type', 'balls', 'strikes',
              'plate_x','plate_z', 'sz_top', 'sz_bot']]
df_keep = df_keep[df_keep['pitch_type'].notna()]
df_keep = df_keep.rename(columns={'estimated_woba_using_speedangle': 'xwoba', 'stand': 'b_stands', 'plate_x': 'plate_horz', 'plate_z': 'plate_vert'})

In [21]:
batter = pd.read_csv('./data/fg_00_24.csv')
batter = batter[['Name', 'MLBAMID']]
batter = batter.reset_index()
batter = batter.drop(columns=['index'])

In [ ]:
df_keep = df_keep.merge(batter, how='left',left_on='batter', right_on='MLBAMID')

In [ ]:
df_keep = df_keep.drop_duplicates()

In [28]:
# Move 'Name' column to be the 4th column and drop 'MLBAMID'
cols = list(df_keep.columns)
cols.remove('Name')
cols.remove('pitcher')
cols.remove('batter')
cols = cols[:3] + ['Name'] + cols[3:]
df_keep = df_keep[cols]

In [29]:
df_keep = df_keep.rename(columns={'player_name': 'pitcher', 'Name':'batter'})

In [35]:
import unicodedata
def remove_accents(name):
    # Remove accents
    name = ''.join(
        c for c in unicodedata.normalize('NFKD', name)
        if not unicodedata.combining(c)
    )
    # Keep only lowercase a-z letters and spaces
    return ''.join(c for c in name if c in 'abcdefghijklmnopqrstuvwxyz ')

In [36]:
df_keep['pitcher'] = df_keep['pitcher'].str.lower()
df_keep['pitcher'] = df_keep['pitcher'].str.split(', ').str[::-1].str.join(' ')
df_keep['pitcher'] = df_keep['pitcher'].apply(remove_accents)

In [37]:
df_keep['batter'] = df_keep['batter'].astype(str).str.lower()
df_keep['batter'] = df_keep['batter'].apply(remove_accents)

In [38]:
df_keep.to_csv('./data/pitch_21_24.csv')